In [1]:
import json
from pathlib import Path
import pickle

import datasets
import numpy as np
import torch
import tqdm
import transformers

In [2]:
DATASET_TUPLE = ("ag_news",)
RETRIEVER_NAME = "facebook/contriever"
OUTPUT_DIR = Path(f"vectors_{'_'.join(DATASET_TUPLE)}_{RETRIEVER_NAME.split('/')[-1]}")
BATCH_SIZE = 100
DEVICE = 2


OUTPUT_DIR.mkdir(exist_ok=True)
with (OUTPUT_DIR / "config.json").open("w") as f:
    json.dump(dict(dataset_tuple=DATASET_TUPLE, retriever_name=RETRIEVER_NAME, dataset_type="single_entry_classification"), f)

In [3]:
dataset = datasets.load_dataset(*DATASET_TUPLE)


Using custom data configuration default
Reusing dataset ag_news (/home/mila/g/gagnonju/.cache/huggingface/datasets/ag_news/default/0.0.0/bc2bcb40336ace1a0374767fc29bb0296cdaf8a6da7298436239c54d79180548)


  0%|          | 0/2 [00:00<?, ?it/s]

In [4]:
samples = [x["text"] for x in dataset["train"]]
labels = [x["label"] for x in dataset["train"]]
with (OUTPUT_DIR / "train_samples.json").open("w") as f:
    json.dump(dict(inputs=samples, labels=labels, ), f, indent=4)


In [5]:
model = transformers.AutoModel.from_pretrained(RETRIEVER_NAME).to(DEVICE)
tokenizer = transformers.AutoTokenizer.from_pretrained(RETRIEVER_NAME)

RuntimeError: CUDA error: out of memory
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

In [ ]:
def batcher(array, size):
    for i in range(0, len(array), size):
        yield array[i : i + size]

all_representations = []
for batch in tqdm.tqdm(batcher(samples, BATCH_SIZE), total=len(samples) // BATCH_SIZE):
    batch_toked = tokenizer.batch_encode_plus(batch, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    with torch.inference_mode():
        representations = model(**batch_toked).pooler_output.detach().cpu().numpy()
    all_representations.append(representations)

NameError: name 'asdasdas' is not defined

In [ ]:
embeddings = np.concatenate(all_representations, axis=0)
with (OUTPUT_DIR / "train_vectors.npy").open("wb") as f:
    np.save(f, embeddings)